In [1]:
import pandas as pd
import numpy as np
data=pd.read_csv("C:\\Users\\91852\\Downloads\\ParsedLogs.csv")
data.head()

,"id,message,@version,@timestamp"
0,"3y51c38BQ2yDyGVTnYjI,kernel audit,1,2022-03-10..."
1,"4C51c38BQ2yDyGVTnYjI,kernel,1,2022-03-10T10:52..."
2,"4S51c38BQ2yDyGVTnYjI,kernel Extensions device ..."
3,"4i51c38BQ2yDyGVTnYjI,kernel tunneling driver,1..."
4,"4y51c38BQ2yDyGVTnYjI,kernel audit,1,2022-03-10..."


In [2]:
df=data['id,message,@version,@timestamp'].str.split(',',expand=True)

In [3]:
df=df[1]

In [4]:
df.head()

0                     kernel audit
1                           kernel
2    kernel Extensions device dblo
3          kernel tunneling driver
4                     kernel audit
Name: 1, dtype: object

In [5]:
df=df.to_frame()

In [6]:
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25567 entries, 0 to 25566
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   1       25567 non-null  object
dtypes: object(1)
memory usage: 199.9+ KB


In [7]:
df[1]

0                                kernel audit
1                                      kernel
2               kernel Extensions device dblo
3                     kernel tunneling driver
4                                kernel audit
                         ...                 
25562                                  kernel
25563                                  kernel
25564         kernel checking image initramfs
25565    kernel Freeing initrd memory k freed
25566               kernel Autodetecting RAID
Name: 1, Length: 25567, dtype: object

In [8]:
messages=df

In [9]:
corpus=set()

In [10]:
for str in df[1]:
    corpus.add(str)

In [11]:
corpus

{'',
 'FreeWnn jserver shutdown failed',
 'FreeWnn jserver shutdown succeeded',
 'IIim htt succeeded',
 'PAMrootok pam_check_access failed proper access system_usystem_rkernel_t',
 'anacron anacron succeeded',
 'apmd Charge unknown',
 'apmd Exiting',
 'apmd Version',
 'apmd apmd shutdown succeeded',
 'apmd apmd succeeded',
 'atd atd shutdown succeeded',
 'atd atd succeeded',
 'autofs automount succeeded',
 'bluetooth hcid succeeded',
 'bluetooth sdpd succeeded',
 'canna Stopping Canna server succeeded',
 'canna succeeded',
 'cannaserver irw_killservercannaserver end',
 'crond crond shutdown succeeded',
 'crond crond succeeded',
 'cup cupsd shutdown succeeded',
 'cup cupsd succeeded',
 'cupslpd Connection cable',
 'cupslpd Unable get command line client',
 'dd record',
 'exiting signal',
 'fingerd Client hung probable portscan',
 'ftpd ANONYMOUS FTP LOGIN FROM',
 'ftpd User unknown timed second',
 'ftpd connection',
 'ftpd getpeername ftpd Transport endpoint connected',
 'ftpd setsockop

In [12]:
words = []
for text in corpus:
    for word in text.split(' '):
        words.append(word)

words = set(words)
words=list(words)

In [13]:
words

['',
 'device',
 'removing',
 'Freeing',
 'reported',
 'journal_startxxe',
 'successfully',
 'class',
 'EBADF',
 'machine',
 'fc',
 'I',
 'III',
 'rhosttroi',
 'rhostunknown',
 'mqueue',
 'cupsd',
 'cable',
 'FS',
 'keyword',
 'htt_server',
 'inodes',
 'Switching',
 'sysfs',
 'fba',
 'Set',
 'A',
 'isaserio',
 'htt',
 'Compressed',
 'do_rw_procxxd',
 'devicemapper',
 'io',
 'seed',
 'xffaxffa',
 'LOGIN',
 'loop',
 'later',
 'M',
 'username',
 'Coppermine',
 'ae',
 'irqbalance',
 'Serial',
 'rhostfparas',
 'do_proc_dointvec_convxx',
 'memory',
 'rhostdialupmum',
 'working',
 'BIOSe',
 'vt_console_printxxf',
 'Service',
 'v',
 'rhostponteix',
 'Failure',
 'rhostsfbbd',
 'waiting',
 'rhosttest',
 'hid',
 'rhostcfc',
 'allocated',
 'National',
 'aeba',
 'ext_reserve_inode_writexx',
 'IO',
 'capability',
 'Setting',
 'rpc_pipefs',
 'shutting',
 'do_swap_pagexdxf',
 'quota',
 'DMI',
 'etcsmartd',
 'status',
 'filemap_nopagexxd',
 'configure',
 'k',
 'bucket',
 'server',
 'rhostned',
 'Receiv

In [14]:
len(words)

928

## Data Generation

### Creating input and label for skip gram

In [15]:
word2int = {}

for i,word in enumerate(words):
    word2int[word] = i

sentences = []
for sentence in corpus:
    sentences.append(sentence.split())
    
WINDOW_SIZE =4 

data = []
for sentence in sentences:
    for idx, word in enumerate(sentence):
        for neighbor in sentence[max(idx - WINDOW_SIZE, 0) : min(idx + WINDOW_SIZE, len(sentence)) + 1] : 
            if neighbor != word:
                data.append([word, neighbor])

In [16]:
df = pd.DataFrame(data, columns = ['input', 'label'])

In [17]:
df.head(10)

,input,label
0,kernel,cfe
1,kernel,generic_file_aio_write_nolockxxe
2,cfe,kernel
3,cfe,generic_file_aio_write_nolockxxe
4,generic_file_aio_write_nolockxxe,kernel
5,generic_file_aio_write_nolockxxe,cfe
6,squid,Squid
7,squid,Parent
8,squid,child
9,squid,process


In [18]:
df.shape

(8564, 2)

## Define tensorflow graph

In [19]:
ONE_HOT_DIM = len(words)

# function to convert numbers to one hot vectors
def to_one_hot_encoding(data_point_index):
    one_hot_encoding = np.zeros(ONE_HOT_DIM)
    one_hot_encoding[data_point_index] = 1
    return one_hot_encoding

X = [] # input word
Y = [] # target word

for x, y in zip(df['input'], df['label']):
    X.append(to_one_hot_encoding(word2int[ x ]))
    Y.append(to_one_hot_encoding(word2int[ y ]))


In [20]:
# convert them to numpy arrays
X_train = np.asarray(X)
Y_train = np.asarray(Y)


In [21]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior() 
# making placeholders for X_train and Y_train
x = tf.placeholder(tf.float32, shape=(None, ONE_HOT_DIM))
y_label = tf.placeholder(tf.float32, shape=(None, ONE_HOT_DIM))

Instructions for updating:
non-resource variables are not supported in the long term


In [22]:
EMBEDDING_DIM = 8

In [23]:
# hidden layer: which represents word vector eventually
W1 = tf.Variable(tf.random_normal([ONE_HOT_DIM, EMBEDDING_DIM]))
b1 = tf.Variable(tf.random_normal([1])) #bias
hidden_layer = tf.add(tf.matmul(x,W1), b1)

# output layer
W2 = tf.Variable(tf.random_normal([EMBEDDING_DIM, ONE_HOT_DIM]))
b2 = tf.Variable(tf.random_normal([1]))
prediction = tf.nn.softmax(tf.add( tf.matmul(hidden_layer, W2), b2))

# loss function: cross entropy
loss = tf.reduce_mean(-tf.reduce_sum(y_label * tf.log(prediction), axis=[1]))

# training operation
train_op = tf.train.GradientDescentOptimizer(0.05).minimize(loss)

## Train

In [24]:
sess = tf.Session()
init = tf.global_variables_initializer()
sess.run(init) 

iteration = 500
for i in range(iteration):
    # input is X_train which is one hot encoded word
    # label is Y_train which is one hot encoded neighbor word
    sess.run(train_op, feed_dict={x: X_train, y_label: Y_train})
    if i % 25 == 0:
        print('iteration '+' loss is : ', sess.run(loss, feed_dict={x: X_train, y_label: Y_train}))

iteration  loss is :  10.206899
iteration  loss is :  10.086233
iteration  loss is :  9.98146
iteration  loss is :  9.887272
iteration  loss is :  9.801764
iteration  loss is :  9.723466
iteration  loss is :  9.65122
iteration  loss is :  9.584104
iteration  loss is :  9.521378
iteration  loss is :  9.462443
iteration  loss is :  9.406816
iteration  loss is :  9.354096
iteration  loss is :  9.303966
iteration  loss is :  9.25616
iteration  loss is :  9.210462
iteration  loss is :  9.166686
iteration  loss is :  9.124684
iteration  loss is :  9.084322
iteration  loss is :  9.045478
iteration  loss is :  9.008044


In [25]:
# Now the hidden layer (W1 + b1) is actually the word look up table
vectors = sess.run(W1 + b1)
print(vectors)

[[-1.34628     1.7185472   0.47879556 ... -0.10000327  1.4526161
  -1.2493275 ]
 [-0.416508   -0.83633506  1.3446383  ...  1.8896505  -1.1901302
  -0.06786615]
 [-0.13090193  1.1108168   0.83876586 ... -0.36790064  0.1474733
  -0.5152575 ]
 ...
 [ 0.8963393  -0.6267369   0.16130355 ... -0.20033833  0.3840023
   0.8323487 ]
 [-1.7561134  -0.09083712  0.9948838  ...  1.0521235  -0.9483279
   0.6685485 ]
 [-0.45985827  0.2619718  -1.6297581  ...  0.0515188   2.0806403
   0.9633503 ]]


## Word Vector table

In [26]:
vec = pd.DataFrame(vectors)

In [27]:
vec

,0,1,2,3,4,5,6,7
0,-1.346280,1.718547,0.478796,1.287899,-0.580093,-0.100003,1.452616,-1.249328
1,-0.416508,-0.836335,1.344638,0.324551,0.478547,1.889650,-1.190130,-0.067866
2,-0.130902,1.110817,0.838766,0.274173,0.661058,-0.367901,0.147473,-0.515257
3,1.990776,0.821354,0.882001,-0.665079,0.593760,0.940245,0.400837,0.048411
4,0.580731,-0.184051,1.364765,0.910252,0.990614,-0.009529,0.322102,0.052917
...,...,...,...,...,...,...,...,...
923,-0.382595,1.097435,-0.289577,-0.208185,0.469941,0.411616,0.153058,1.668721
924,1.808834,0.765397,-0.288206,-0.799263,0.142074,-0.659322,0.784421,0.980223
925,0.896339,-0.626737,0.161304,-0.416884,0.692747,-0.200338,0.384002,0.832349
926,-1.756113,-0.090837,0.994884,0.827878,0.312514,1.052124,-0.948328,0.668549


# Preapring input for clustering

In [28]:
messages.head()

,1
0,kernel audit
1,kernel
2,kernel Extensions device dblo
3,kernel tunneling driver
4,kernel audit


In [29]:
messages.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25567 entries, 0 to 25566
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   1       25567 non-null  object
dtypes: object(1)
memory usage: 199.9+ KB


In [69]:
sentences=[]
for str in messages[1]:
    sentences.append(str)

In [70]:
type(sentences[0])

str

In [71]:
print(type(words))
l=len(words)
print(l)

<class 'list'>
928


In [72]:
dict_words={}

In [73]:
for i in range(l):
    dict_words[words[i]]=list(vec.iloc[i,:])


In [74]:
print(type(dict_words["end"]))

<class 'list'>


### Calculating mean of vectors

In [122]:
list_final=[]
for text in sentences:
    temp=text.split(' ')
    lt=[]
    for s in temp:
        lt.append(dict_words[s])
    npat=np.array(lt)
    t=np.mean(npat,axis=0).tolist()
    list_final.append(t)

In [123]:
list_final

[[0.6502128019928932,
  -0.38197506964206696,
  -0.5226287096738815,
  0.2428198605775833,
  1.0918804854154587,
  0.46420952677726746,
  0.3054429441690445,
  0.24906538426876068],
 [0.19944949448108673,
  -0.06546655297279358,
  -0.035290807485580444,
  -0.03891727328300476,
  -0.008559852838516235,
  -0.0765426754951477,
  0.3534378409385681,
  0.05378177762031555],
 [0.5093078427016735,
  -0.48438483476638794,
  1.213730551302433,
  0.24422059208154678,
  0.514459915459156,
  0.24127133190631866,
  0.045391686260700226,
  0.34796591848134995],
 [-0.014745468894640604,
  0.6595817804336548,
  -0.31156351168950397,
  0.7820300360520681,
  0.43928245703379315,
  -0.008672922849655151,
  -0.3499445617198944,
  -0.6382402777671814],
 [0.6502128019928932,
  -0.38197506964206696,
  -0.5226287096738815,
  0.2428198605775833,
  1.0918804854154587,
  0.46420952677726746,
  0.3054429441690445,
  0.24906538426876068],
 [0.6502128019928932,
  -0.38197506964206696,
  -0.5226287096738815,
  0.242

In [124]:
cl_input=pd.DataFrame(list_final)

In [125]:
cl_input

,0,1,2,3,4,5,6,7
0,0.650213,-0.381975,-0.522629,0.242820,1.091880,0.464210,0.305443,0.249065
1,0.199449,-0.065467,-0.035291,-0.038917,-0.008560,-0.076543,0.353438,0.053782
2,0.509308,-0.484385,1.213731,0.244221,0.514460,0.241271,0.045392,0.347966
3,-0.014745,0.659582,-0.311564,0.782030,0.439282,-0.008673,-0.349945,-0.638240
4,0.650213,-0.381975,-0.522629,0.242820,1.091880,0.464210,0.305443,0.249065
...,...,...,...,...,...,...,...,...
25562,0.199449,-0.065467,-0.035291,-0.038917,-0.008560,-0.076543,0.353438,0.053782
25563,0.199449,-0.065467,-0.035291,-0.038917,-0.008560,-0.076543,0.353438,0.053782
25564,0.969946,-0.705862,0.413066,0.361516,0.294271,0.391053,0.324403,0.215872
25565,0.777869,0.637458,0.145324,0.088151,0.334305,0.712134,0.057618,0.096425


In [126]:
cl_input.to_csv(r'C:\Users\91852\OneDrive\Desktop\BMC\cl_input.csv', index=False)

# Clustering-DBSCAN